# Set Up

In [5]:
%pwd


'/pscratch/sd/m/mansisak/memorization/figs'

In [ ]:
!conda list

In [6]:
!module load conda

In [7]:
!conda activate /pscratch/sd/m/mansisak/memorization/env/

In [24]:
!ls /pscratch/sd/m/mansisak/memorization/model_ckpts/EleutherAI_edit/

act	localization_results_108000.csv  mem_seq_pythia-2.8b-deduped
attrib	localization_results_143000.csv  mem_seq_pythia-6.9b-deduped
greedy	localization_results_36000.csv	 slim
hc	localization_results_72000.csv


In [ ]:
ls 

In [45]:
import pandas as pd
import numpy as np

# Load in Results

In [57]:
result_path = "/pscratch/sd/m/mansisak/memorization/model_ckpts/EleutherAI_edit/"

file_name = "localization_results_72000.csv"

df = pd.read_csv(f'{result_path}{file_name}')

base_stats = df.loc[0].copy()
df = df.drop_duplicates()
df.head()

,model_name,localization_method,ratio,batch_size,epochs,lambda_l1,stop_loss,lr,prompt_len,ig_steps,momentum,weight_decay,step,assess_mem,seed,model_path,results_path,perc,perp,unlearn_set_name
0,EleutherAI/pythia-6.9b-deduped,act,0.001,32,1,1000,0.1,0.1,32,1,0.9,0.0005,72000,0,0,../../model_ckpts/EleutherAI/pythia-6.9b-deduped,../../model_ckpts/EleutherAI_edit/localization...,0.564356,20.980392,NaN
1,EleutherAI/pythia-6.9b-deduped,act,0.001,32,1,1000,0.1,0.1,32,1,0.9,0.0005,72000,0,0,../../model_ckpts/EleutherAI/pythia-6.9b-deduped,../../model_ckpts/EleutherAI_edit/localization...,0.281188,21.607843,mem
2,EleutherAI/pythia-6.9b-deduped,act,0.010,32,1,1000,0.1,0.1,32,1,0.9,0.0005,72000,0,0,../../model_ckpts/EleutherAI/pythia-6.9b-deduped,../../model_ckpts/EleutherAI_edit/localization...,0.164356,23.372549,mem
3,EleutherAI/pythia-6.9b-deduped,act,0.050,32,1,1000,0.1,0.1,32,1,0.9,0.0005,72000,0,0,../../model_ckpts/EleutherAI/pythia-6.9b-deduped,../../model_ckpts/EleutherAI_edit/localization...,0.108911,25.411765,mem
4,EleutherAI/pythia-6.9b-deduped,hc,0.001,32,1,1000,0.1,0.1,32,1,0.9,0.0005,72000,0,0,../../model_ckpts/EleutherAI/pythia-6.9b-deduped,../../model_ckpts/EleutherAI_edit/localization...,0.512871,21.568627,mem


In [58]:
df[df['unlearn_set_name'].isna()]

,model_name,localization_method,ratio,batch_size,epochs,lambda_l1,stop_loss,lr,prompt_len,ig_steps,momentum,weight_decay,step,assess_mem,seed,model_path,results_path,perc,perp,unlearn_set_name
0,EleutherAI/pythia-6.9b-deduped,act,0.001,32,1,1000,0.1,0.1,32,1,0.9,0.0005,72000,0,0,../../model_ckpts/EleutherAI/pythia-6.9b-deduped,../../model_ckpts/EleutherAI_edit/localization...,0.564356,20.980392,NaN
24,EleutherAI/pythia-2.8b-deduped,act,0.001,32,1,1000,0.1,0.1,32,1,0.9,0.0005,72000,0,1,../../model_ckpts/EleutherAI/pythia-2.8b-deduped,../../model_ckpts/EleutherAI_edit/localization...,0.372277,23.509804,NaN
26,EleutherAI/pythia-2.8b-deduped,act,0.001,32,1,1000,0.1,0.1,32,1,0.9,0.0005,72000,0,0,../../model_ckpts/EleutherAI/pythia-2.8b-deduped,../../model_ckpts/EleutherAI_edit/localization...,0.372277,23.509804,NaN
28,EleutherAI/pythia-2.8b-deduped,act,0.001,32,1,1000,0.1,0.1,32,1,0.9,0.0005,72000,0,3,../../model_ckpts/EleutherAI/pythia-2.8b-deduped,../../model_ckpts/EleutherAI_edit/localization...,0.372277,23.509804,NaN


In [62]:
import statistics
import math
def return_ideal_method_for_lang(base_early_stats, weight_early, backdoor=False):
  #desired columns
  cols = ["perc", "perp",'localization_method','ratio']

  #Convert base stats to a df
  base_frame = base_early_stats.to_frame().T[cols]

  #Generate Percent Differences
  weight_early["perc_diff"] =  (weight_early['perc'] - base_early_stats['perc']) /  base_early_stats['perc']


  weight_early["perp_diff"] = (weight_early['perp'] - base_early_stats['perp']) /  base_early_stats['perp']

  weight_early["score"] = 0 # zero initialize -- we will populate it below

  #Compute Scores
  for index, row in weight_early.iterrows():


    if row['unlearn_set_name'] == "mem":
        if row['perc_diff'] == 0:
            weight_early.loc[index, "score"] += 100 # this is how we penalize zero perc drop in mem
        weight_early.loc[index, "score"] += (row['perc_diff'])



        weight_early.loc[index, "score"] += statistics.mean([row['perp_diff']])

  df = pd.DataFrame(columns=cols)
  #append base stats to df
  base_frame['localization_method'] = "BASE_STATS"
  df = pd.concat([df, base_frame])

  #Print out ideal method for each dist based on the min score
  for unlearn_set in ['greedy', 'durable', 'durable_agg', 'random', 'random_greedy', "hc", "slim", 'act', ]:
    #print(unlearn_set)
    subset = weight_early.loc[weight_early['localization_method'] == unlearn_set]
    if subset.shape[0] == 0:
      print("no results yet for: ", unlearn_set)
      continue
    sub_frame = subset[subset.score == subset.score.min()].drop_duplicates()
    sub_frame = sub_frame[cols]
    #print(sub_frame)
    df = pd.concat([df, sub_frame])

  df[["perc",'ratio',]] *= 100

  return df.drop_duplicates()


In [63]:
result_path = "/pscratch/sd/m/mansisak/memorization/model_ckpts/EleutherAI_edit/"
for model_name in ['EleutherAI/pythia-6.9b-deduped', 'EleutherAI/pythia-2.8b-deduped']:
    for step in [36000, 72000, 108000, 143000]:
        print("STEP: ", step, " model_name:  ", model_name)
        file_name = f"localization_results_{step}.csv"

        df = pd.read_csv(f'{result_path}{file_name}')
        df = df[df['model_name'] == model_name]
        #base_stats = df.loc[0].copy()
        base_stats = df[df['unlearn_set_name'].isna()].iloc[0]
        df = df.drop_duplicates()

        df = return_ideal_method_for_lang(base_stats, df, backdoor=False)
        print(df)

STEP:  36000  model_name:   EleutherAI/pythia-6.9b-deduped
no results yet for:  durable
no results yet for:  durable_agg
no results yet for:  random
no results yet for:  random_greedy
          perc       perp localization_method  ratio
0     42.17822  22.901961          BASE_STATS    0.1
128   4.950495  27.803922              greedy  0.001
12   13.663366  27.352941                  hc    5.0
19   37.029704  23.254902                slim    5.0
3    11.485149  26.843137                 act    5.0
STEP:  72000  model_name:   EleutherAI/pythia-6.9b-deduped
no results yet for:  durable
no results yet for:  durable_agg
no results yet for:  random
no results yet for:  random_greedy
          perc       perp localization_method  ratio
0    56.435645  20.980392          BASE_STATS    0.1
130   4.356436  27.647059              greedy  0.001
11   15.841585  26.078431                  hc    5.0
19   47.524753  21.431373                slim    5.0
3     10.89109  25.411765                 act    

/tmp/ipykernel_1699323/995005519.py:25: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-0.4178403681229723' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  weight_early.loc[index, "score"] += (row['perc_diff'])
/tmp/ipykernel_1699323/995005519.py:25: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-0.5017543748475181' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  weight_early.loc[index, "score"] += (row['perc_diff'])
/tmp/ipykernel_1699323/995005519.py:25: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-0.5950413373906054' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  weight_early.loc[index, "score"] += (row['perc_diff'])
/tmp/ipykernel_1699323/99500551

          perc       perp localization_method  ratio
24   37.227723  23.509804          BASE_STATS    0.1
127   2.772277  31.901961              greedy  0.001
53    5.148515  33.509804                  hc    5.0
94   26.138616  24.333333                slim    5.0
30   11.683169  25.901961                 act    1.0
STEP:  108000  model_name:   EleutherAI/pythia-2.8b-deduped
no results yet for:  durable
no results yet for:  durable_agg
no results yet for:  random
no results yet for:  random_greedy
          perc       perp localization_method  ratio
22   48.118815  22.333333          BASE_STATS    0.1
128   3.960396  31.862745              greedy  0.001
60   14.851485   28.54902                  hc    5.0
81   29.504952  23.352941                slim    5.0
30   15.841585  24.215686                 act    1.0
STEP:  143000  model_name:   EleutherAI/pythia-2.8b-deduped
no results yet for:  durable
no results yet for:  durable_agg
no results yet for:  random
no results yet for:  random_g

/tmp/ipykernel_1699323/995005519.py:25: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-0.4962963198315451' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  weight_early.loc[index, "score"] += (row['perc_diff'])


# Git commit

In [29]:
!git add /pscratch/sd/m/mansisak/memorization/figs/pythia_unlearning_results.ipynb
!git commit -m "updated unlearning visualization/analysis"

[WARNING] Unstaged files detected.
[INFO] Stashing unstaged files to /global/u1/m/mansisak/.cache/pre-commit/patch1723567698-1732221.
black................................................(no files to check)Skipped
[INFO] Restored changes from /global/u1/m/mansisak/.cache/pre-commit/patch1723567698-1732221.
[main 3cfc5c7] updated unlearning visualization/analysis
 1 file changed, 92 insertions(+)
